## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1 Import the WeatherPy_database.csv file.
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")

In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
city_data_user_tem_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp)&(city_data_df["Max Temp"] <= max_temp)]
city_data_user_tem_df.head()             

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
1,1,Ouesso,CG,1.61,16.05,79.90,84,72,1.86,light rain
11,11,Cockburn Town,TC,21.46,-71.14,80.10,80,100,18.16,moderate rain
13,13,Puerto Carreno,CO,6.19,-67.49,88.77,61,67,4.18,broken clouds
17,17,Dhidhdhoo,MV,6.88,73.10,84.00,76,100,12.44,light rain
23,23,Poum,NC,-20.23,164.02,76.86,73,0,16.84,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
print((city_data_user_tem_df).count())
city_data_user_tem_df["Max Temp"].replace('', np.nan, inplace = True)
           
city_data_user_tem_df

city_data_user_tem_df.count()

City_ID                210
City                   210
Country                209
Lat                    210
Lng                    210
Max Temp               210
Humidity               210
Cloudiness             210
Wind Speed             210
Weather Description    210
dtype: int64


C:\Users\aksha\anaconda3\envs\PythonData\lib\site-packages\pandas\core\generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


City_ID                210
City                   210
Country                209
Lat                    210
Lng                    210
Max Temp               210
Humidity               210
Cloudiness             210
Wind Speed             210
Weather Description    210
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
new_city_data_df = city_data_user_tem_df.dropna()
new_city_data_df.count()

City_ID                209
City                   209
Country                209
Lat                    209
Lng                    209
Max Temp               209
Humidity               209
Cloudiness             209
Wind Speed             209
Weather Description    209
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = new_city_data_df[["City", "Country", "Max Temp",'Weather Description', "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Ouesso,CG,79.90,light rain,1.61,16.05,
11,Cockburn Town,TC,80.10,moderate rain,21.46,-71.14,
13,Puerto Carreno,CO,88.77,broken clouds,6.19,-67.49,
17,Dhidhdhoo,MV,84.00,light rain,6.88,73.10,
23,Poum,NC,76.86,clear sky,-20.23,164.02,
26,Victoria,HK,79.00,clear sky,22.29,114.16,
27,Banda Aceh,ID,79.18,heavy intensity rain,5.56,95.32,
31,Atuona,PF,77.27,broken clouds,-9.80,-139.03,
33,Georgetown,MY,80.60,heavy intensity rain,5.41,100.34,
37,San Cristobal,VE,80.60,fog,7.77,-72.22,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df.head()        
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Ouesso,CG,79.90,light rain,1.61,16.05,Hôtel ABENI
11,Cockburn Town,TC,80.10,moderate rain,21.46,-71.14,Osprey Beach Hotel
13,Puerto Carreno,CO,88.77,broken clouds,6.19,-67.49,Casa Hotel Victoria
17,Dhidhdhoo,MV,84.00,light rain,6.88,73.10,Dhidhoo beach
23,Poum,NC,76.86,clear sky,-20.23,164.02,Gîte kajeon


In [19]:
# 7. Drop the rows where there is no Hotel Name.

# Replace Hotel name with nan where no data is found
print((hotel_df).count())
hotel_df["Hotel Name"].replace('', np.nan, inplace = True)
hotel_df.count()

 # Drop the nan rows
hotel_df.dropna(subset=['Hotel Name'], inplace = True)
print(hotel_df.head(20))
print((hotel_df).count())


City                   193
Country                193
Max Temp               193
Weather Description    193
Lat                    193
Lng                    193
Hotel Name             193
dtype: int64
              City Country  Max Temp   Weather Description    Lat     Lng  \
1           Ouesso      CG     79.90            light rain   1.61   16.05   
11   Cockburn Town      TC     80.10         moderate rain  21.46  -71.14   
13  Puerto Carreno      CO     88.77         broken clouds   6.19  -67.49   
17       Dhidhdhoo      MV     84.00            light rain   6.88   73.10   
23            Poum      NC     76.86             clear sky -20.23  164.02   
26        Victoria      HK     79.00             clear sky  22.29  114.16   
27      Banda Aceh      ID     79.18  heavy intensity rain   5.56   95.32   
31          Atuona      PF     77.27         broken clouds  -9.80 -139.03   
33      Georgetown      MY     80.60  heavy intensity rain   5.41  100.34   
37   San Cristobal      VE  

In [15]:
#hotel_df["Hotel Name"].isna().unique()

In [20]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")


In [29]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description} and{Max Temp} °F </dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [30]:
# 11a. Add a marker layer for each city to the map. 
locations =  hotel_df[["Lat", "Lng"]]
max_temp =  hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))